In [2]:
from enum import IntEnum
import glob
import json
import time

class LegiscanStatus(IntEnum):
    NA = 0
    Introduced = 1
    Engrossed = 2
    Enrolled = 3
    Passed = 4
    Vetoed = 5
    Failed = 6
    Override = 7
    Chaptered = 8
    Refer = 9
    Report_Pass = 10
    Report_DNP = 11
    Draft = 12

def get_meta(meta_name):
    with open(meta_name, 'r') as f:
        return json.load(f)
    
def summarize(meta):
    bill = meta['bill']
    
    return {
        'state': bill['state'],
        'bill_id': bill['bill_number'],
        'status_date': bill['status_date'],
        'legiscan_bill_id': str(bill['bill_id']),
        'status': str(LegiscanStatus(bill['status'])).split('.')[1],
        'sponsors': [sponsor['name'] for sponsor in bill['sponsors']],
        'title': bill['title'],
        'description': bill['description'],
        'doc_id': str(sorted(
            bill['texts'], 
            key=lambda t: t['date'], 
            reverse=True
        )[0]['doc_id']) if bill['texts'] else None,
    }

start = time.time()

summaries = [
    summarize(get_meta(meta_name)) 
    for meta_name
    in glob.glob('../tmp/*meta*.json')
]

with open('../artifacts/aggregate.json', 'w') as f:
    json.dump(summaries, f, indent=2)
    
end = time.time()

print(f'Summarized {len(summaries)} items ({(end-start):.2f}s elapsed)')

Summarized 481 items (0.10s elapsed)
